In [1]:
import layers
import modules
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from scipy.signal import resample_poly
from pathlib import Path
from sklearn.model_selection import train_test_split
import os

from AF_metrics_utils import *

ModuleNotFoundError: No module named 'layers'

In [ ]:
FS_ECG_ORIG = 2048        # Hz
FS_TARGET   = 284         # Hz
WIN_SEC     = 3.75        # window length (s)
STEP_SEC    = 1.875       # 50 % overlap

UP, DOWN = FS_TARGET, FS_ECG_ORIG          # 71 / 512 after gcd

def load_record(rec_idx: int, env_dir, ecg_dir):
    """Load one record and return 284 Hz ECG + envelope arrays."""
    env  = sio.loadmat(env_dir / f"{rec_idx}_envelope.mat")["x_up"].squeeze()
    ecg  = sio.loadmat(ecg_dir / f"results01_{rec_idx}.mat")["fref1"].flatten()
    ecg284 = resample_poly(ecg, UP, DOWN).astype(np.float32)
    env    = env.astype(np.float32)
    # Trim for minor difference
    L = min(len(ecg284), len(env))
    return ecg284[:L], env[:L]

In [22]:
WIN = int(WIN_SEC  * FS_TARGET)      # 3.75 s × 284 Hz = 1065
STEP = int(STEP_SEC * FS_TARGET)     # 50 % overlap

X, Y = [], []
for rec in range(1, 61):                
    ecg284, env = load_record(rec,
                              Path("/home/tsu25/envelops"),
                              Path("/labs/katebilab/NinFEA_Data/Processed_Data"))
    for s in range(0, len(env) - WIN, STEP):
        ecg_win = ecg284[s:s+WIN]
        env_win = env[s:s+WIN]

        # per-window normalisation
        ecg_win = (ecg_win - ecg_win.mean()) / (ecg_win.std() + 1e-6)
        env_min, env_ptp = env_win.min(), env_win.ptp()
        env_win = 2*(env_win - env_min)/(env_ptp + 1e-6) - 1  # [-1, 1]

        X.append(ecg_win[:, None])    # add channel dim
        Y.append(env_win)             # (WIN,)  or  env_win[:, None]

X = np.asarray(X, dtype=np.float32)
Y = np.asarray(Y, dtype=np.float32)
print("Final shapes:", X.shape, Y.shape)   # (total_windows, 1065, 1) …

Final shapes: (885, 1065, 1) (885, 1065)
